## Задание: Предсказать вероятность того, что твит получит более 20 ретвитов за первые 48 часов.

## Ход выполнения исследования:
1. Импорт данных
2. Создания функции для обработки датасета и генерации новых признаков:
    * clear_text - текс твита без url, хэштегов и "упоминаний"
    * clear_text_with_tags - текс твита без url, знака # и "упоминаний"
    
    * is_reply - является ли твит ответом
    
    * user_reg_days - сколько дней пользователь уже зарегистрирован (отсчёт от макс. даты регистрации)
    * count_reference - сколько "упоминаний" в твите
    * count_url - сколько url в твите
    * count_hashteg - сколько хэштегов в твите
    
    * user_discr_len - длина описания юзера
    * text_len - длина твита (исходного текста)
    * clear_text_with_tags_len - длина твита (clear_text_with_tags)
    * clear_text_len - длина твита (clear_text)

    * is_en - является ли язык пользователя английским
    * is_claim - есть ли в твите восклицательный знак
    * is_question - есть ли в твите вопросительный знак
3. Создание NLP модели для предсказания вероятности твита на основе текста твита
4. Предсказание (используя кросс-валидацию) вероятности ретвита на основе NLP модели - добавление признака nlp_prediction
5. Создание модели Random Forest
6. Подбор гиперпараметров с использованием Grid Search (используя кросс-валидацию)
7. Кросс-валидация модели
8. Определение важности признаков
9. Построение и запись прогноза для тестового датасета

In [1]:
from __future__ import division, unicode_literals

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score

## 1. Импорт данных

In [2]:
train = pd.read_csv('train.csv', encoding='utf-8', lineterminator='\n')
test = pd.read_csv('test.csv', encoding='utf-8', lineterminator='\n')

In [3]:
train.head(2)

,id,text,in_reply_to_user_id,user.id,user.name,user.screen_name,user.description,user.location,user.lang,user.time_zone,user.utc_offset,user.statuses_count,user.followers_count,user.friends_count,user.favourites_count,user.created_at,user.geo_enabled,user.is_translation_enabled,user.listed_count,retweet_count
0,629692043326062592,Me and ma fwends 🍎 http://t.co/B3YJ31hZuc,0.0,133840449,Paul C. wilson,xcaptainpaulx,Guitarist for @chunknocaptainc // Founder @off...,NaN,fr,Paris,7200.0,1606,9164,205,758,Fri Apr 16 19:33:58 +0000 2010,True,False,27,12
1,629692041362968576,@SinedioMD @AspyrMedia Try this link https://t...,7629232.0,21245956,Direct2Drive.com,Direct2Drive,D2D has an ever-expanding library of downloada...,"California, USA",en,Pacific Time (US & Canada),-25200.0,3004,7484,708,44,Wed Feb 18 21:58:04 +0000 2009,False,False,330,0


In [4]:
assert(all(train.columns[:-1] == test.columns))
assert(train.columns[-1] == 'retweet_count')

## Описание данных

In [5]:
train['label'] = train['retweet_count'] > 20
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67211 entries, 0 to 67210
Data columns (total 21 columns):
id                             67211 non-null int64
text                           67211 non-null object
in_reply_to_user_id            67211 non-null float64
user.id                        67211 non-null int64
user.name                      67211 non-null object
user.screen_name               67211 non-null object
user.description               66906 non-null object
user.location                  55676 non-null object
user.lang                      67211 non-null object
user.time_zone                 67211 non-null object
user.utc_offset                67211 non-null float64
user.statuses_count            67211 non-null int64
user.followers_count           67211 non-null int64
user.friends_count             67211 non-null int64
user.favourites_count          67211 non-null int64
user.created_at                67211 non-null object
user.geo_enabled               67211 non-null b

Колонки являются урезанным набором полей объекта-сообщения, предоставляемого Twitter API: https://dev.twitter.com/overview/api/tweets.

## 2. Создания функции для обработки датасета и генерации новых признаков:

In [6]:
import re

def prepare_df(df):
    
    # удалим идентификаторы юзера, в связи с условиями задачи
    df.drop(['user.name', 'user.id', 'user.screen_name'] , axis = 1, inplace = True)
    # удалим time_zone, т.к. признак utc_offset лучше описывает сущность
    df.drop('user.time_zone', axis = 1, inplace = True)
    # удалим location из-за большого кол-ва категорий (в том числе с близкой частотой)
    df.drop('user.location', axis = 1, inplace = True)
    # переведём время создания пользователя в datetime
    df['user.created_at'] = pd.to_datetime(df['user.created_at'],format = '%a %b %d %X +0000 %Y')
    # переведём user.utc_offset в часы
    df['user.utc_offset'] = (df['user.utc_offset'] / 3600).astype('int')
    # добавим признак является ли твит ответом
    df['is_reply'] = df['in_reply_to_user_id'] != 0
    df.drop('in_reply_to_user_id', axis = 1, inplace = True)
    # добавим признак является ли язык пользователя английским (как само часто встречающийся)
    df['user.lang'] = df['user.lang'].str.slice(stop = 2)
    df['is_en'] = df['user.lang'] == 'en'
    df.drop('user.lang', axis = 1, inplace = True)
    
    # функция для удаления хэштегов, "упоминаний", ссылок
    def clear_text(post):
        post = re.sub("(@.\w+|#.\w+|(?:https?|ftp):\/\/[\n\S]+)", '', post)
        post = post.strip() #trim
        post = post.lstrip() #ltrim
        post = post.rstrip() #rtrim
        return post
    # функция для удаления "упоминаний", ссылок и знаков "#"
    def clear_text_with_tags(post):
        post = re.sub("(@.\w+|#|(?:https?|ftp):\/\/[\n\S]+)", '', post)
        post = post.strip() #trim
        post = post.lstrip() #ltrim
        post = post.rstrip() #rtrim
        return post
    # функция для подсчёта  хэштегов
    def count_hashteg(post):
        return len(re.findall(r'#.\w+', post))
    # функция для подсчёта  "обрашений"
    def count_reference(post):
        return len(re.findall(r'@.\w+', post))
    # функция для подсчёта  ссылок
    def count_url(post):
        return len(re.findall(r'(?:https?|ftp):\/\/[\n\S]+', post))
    # функция для поиска знаков вопросов
    def is_question(post):
        if post.count('?') > 0:
            return True
        else:
            return False
    # функция для поиска восклицательных знаков
    def is_claim(post):
        if post.count('!') > 0:
            return True
        else:
            return False
        
    # применим написанные функции к тексту постов
    functions = [clear_text, clear_text_with_tags, count_hashteg, count_reference, count_url, is_question, is_claim]
    for fun in functions:
        df[fun.__name__] = df['text'].apply(fun)
        
    # длина поста
    df['text_len'] = df['text'].apply(lambda x: len(x))
    df['clear_text_len'] = df['clear_text'].apply(lambda x: len(x))
    df['clear_text_with_tags_len'] = df['clear_text_with_tags'].apply(lambda x: len(x))    
    # длина описания юзера
    df['user.description'].fillna('', inplace = True)
    df['user_discr_len'] = df['user.description'].apply(lambda x: len(x))
    # сколько дней пользователь уже зарегистрирован (по сравнению с самым недавно зарегистрированным)
    df['user_reg_days'] = df['user.created_at'].max() - df['user.created_at']
    df['user_reg_days'] = (df['user_reg_days'].apply(lambda x: x.total_seconds()/(60*60*24))).astype('int')
    # удалим неиспользуемые столбцы и установим индекс
    df.drop(['user.created_at', 'clear_text','text', 'user.description'], axis = 1, inplace = True)
    df.set_index('id', inplace = True)
    
    return df

## Обработка train и test датасетов

In [7]:
train = prepare_df(train)
test = prepare_df(test)

In [8]:
train.iloc[:2, :11].head(2)

,user.utc_offset,user.statuses_count,user.followers_count,user.friends_count,user.favourites_count,user.geo_enabled,user.is_translation_enabled,user.listed_count,retweet_count,label,is_reply
id,,,,,,,,,,,
629692043326062592,2,1606,9164,205,758,True,False,27,12,False,False
629692041362968576,-7,3004,7484,708,44,False,False,330,0,False,True


In [9]:
train.iloc[:2, 11:].head(2)

,is_en,clear_text_with_tags,count_hashteg,count_reference,count_url,is_question,is_claim,text_len,clear_text_len,clear_text_with_tags_len,user_discr_len,user_reg_days
id,,,,,,,,,,,,
629692043326062592,False,Me and ma fwends 🍎,0,0,1,False,False,41,18,18,82,1934
629692041362968576,True,Try this link,0,2,1,False,False,61,13,13,124,2356


## 3. Создание NLP модели для предсказания вероятности твита на основе текста твита

In [10]:
import nltk
import gensim

tokenizer = nltk.tokenize.TreebankWordTokenizer()

from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()

In [11]:
corpora = [train['clear_text_with_tags'][id] for id in train.index]

In [12]:
target = [train['label'][id] for id in train.index]

In [13]:
from tqdm import tqdm
corpora_tokenzied = []
for doc in tqdm(corpora):
    corpora_tokenzied.append(tokenizer.tokenize(doc.lower()))
    
corpora_stemmed = []

for doc in tqdm(corpora_tokenzied):
    stemmed_doc = [stemmer.stem(token) for token in doc]
    corpora_stemmed.append(stemmed_doc)

dictionary = gensim.corpora.Dictionary(corpora_stemmed)
corpora_bow = [dictionary.doc2bow(doc) for doc in corpora_stemmed]

100%|██████████| 67211/67211 [00:17<00:00, 3897.81it/s]


In [14]:
tfidf = gensim.models.TfidfModel(corpora_bow)
corpora_tfidf = tfidf[corpora_bow]

In [15]:
from scipy.sparse import csc_matrix

data = []
col = []
row = []
for i in tqdm(range(len(corpora_tfidf))):
    for j in range(len(corpora_tfidf[i])):
        data.append(corpora_tfidf[i][j][1])
        col.append(corpora_tfidf[i][j][0])
        row.append(i)

100%|██████████| 67211/67211 [01:15<00:00, 888.59it/s]  | 69/67211 [00:00<01:38, 680.77it/s]


In [16]:
matrix = csc_matrix((data, (row, col)), shape=(len(corpora_tfidf), len(dictionary)))

## 4. Предсказание (используя кросс-валидацию) вероятности ретвита на основе NLP модели - добавление признака nlp_prediction

In [17]:
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn.cross_validation import KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

clsf = LogisticRegression(class_weight = 'balanced')

kf = KFold(matrix.shape[0], n_folds=5, random_state=1)

nlp_auc = cross_val_score(clsf, matrix, target, cv=kf, scoring='roc_auc')
print(np.mean(nlp_auc))
print(nlp_auc)

predictions = cross_val_predict(clsf, matrix, target)
train['nlp_prediction'] = predictions

/home/adam/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0.737818295325
[ 0.70649573  0.72863835  0.76862728  0.74931678  0.73601334]


## Классификатор на всём train

In [18]:
classifier = LogisticRegression(class_weight = 'balanced')
classifier.fit(matrix, target)

def classify(text):
    tokens = [stemmer.stem(token) for token in tokenizer.tokenize(text.lower())]
    bow = dictionary.doc2bow(tokens)
    tfidf_vec = tfidf[bow]
    data = []
    col = []
    row = []
    for token in tfidf_vec:
        data.append(token[1])
        col.append(token[0])
        row.append(0)
    matrix_1 = csc_matrix((data, (row, col)), shape=(1, len(dictionary)))
    return(classifier.predict_proba(matrix_1)[0][1])

## Такой твит скорее всего репостнут

In [19]:
classify("Obama sing a New Year song")

0.89752748081149392

In [20]:
classify("A new war begin")

0.91470163657395531

## А такой нет

In [21]:
classify("the weather is fine")

0.1209713593909261

In [22]:
classify("I am home")

0.24312015201766238

## 5. Создание модели Random Forest

In [23]:
train.columns.values

array(['user.utc_offset', 'user.statuses_count', 'user.followers_count',
       'user.friends_count', 'user.favourites_count', 'user.geo_enabled',
       'user.is_translation_enabled', 'user.listed_count', 'retweet_count',
       'label', 'is_reply', 'is_en', 'clear_text_with_tags',
       'count_hashteg', 'count_reference', 'count_url', 'is_question',
       'is_claim', 'text_len', 'clear_text_len',
       'clear_text_with_tags_len', 'user_discr_len', 'user_reg_days',
       'nlp_prediction'], dtype=object)

In [24]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import cross_val_predict
from sklearn.cross_validation import KFold

In [25]:
features = ['user.utc_offset', 'user.statuses_count', 'user.followers_count',
            'user.friends_count', 'user.favourites_count', 'user.geo_enabled',
            'user.is_translation_enabled', 'user.listed_count','is_reply', 'is_en',
            'count_hashteg', 'count_reference', 'count_url', 'is_question',
            'is_claim', 'text_len', 'clear_text_len', 'clear_text_with_tags_len',
            'user_discr_len', 'user_reg_days','nlp_prediction']

target = ['label']
#target = ['retweet_count']

In [26]:
model = RandomForestClassifier(n_estimators = 10, random_state = 1)

kf = KFold(len(train), n_folds=5, random_state=1)

## 6. Подбор гиперпараметров с использованием Grid Search (используя кросс-валидацию)

In [27]:
from sklearn import cross_validation, grid_search, metrics

In [28]:
parameters_grid = {
    'max_features' : np.arange(5,len(features)),
    'max_depth' : np.arange(3,7,1),
    'min_samples_split' : np.arange(5,30,1),
    'min_samples_leaf' : np.arange(3,10,1),
    'class_weight' : ['balanced']
}

In [29]:
randomized_grid_cv = grid_search.RandomizedSearchCV(model, parameters_grid, scoring = 'roc_auc', cv = kf,
                                                    n_iter = 100, random_state = 0)

In [30]:
%%time
randomized_grid_cv.fit(train[features], train[target])

CPU times: user 8min 44s, sys: 3.08 s, total: 8min 48s
Wall time: 8min 49s


RandomizedSearchCV(cv=sklearn.cross_validation.KFold(n=67211, n_folds=5, shuffle=False, random_state=1),
          error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=1,
            verbose=0, warm_start=False),
          fit_params={}, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'min_samples_leaf': array([3, 4, 5, 6, 7, 8, 9]), 'max_features': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]), 'class_weight': ['balanced'], 'max_depth': array([3, 4, 5, 6]), 'min_samples_split': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29])},
          pre_dispatch='2*n_jobs',

In [31]:
randomized_grid_cv.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=6, max_features=13,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=6, min_samples_split=23,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [32]:
print(randomized_grid_cv.best_score_)
print(randomized_grid_cv.best_params_)

0.9188069614901425
{'class_weight': 'balanced', 'min_samples_leaf': 6, 'min_samples_split': 23, 'max_features': 13, 'max_depth': 6}


## Модель с подобранными гиперпараметрами (и увеличинным n_estimators = 200)

In [33]:
model = RandomForestClassifier(n_estimators = 200, max_features = 13, class_weight = 'balanced', min_samples_split = 23,
                               max_depth = 6, min_samples_leaf = 6, random_state = 1)

In [34]:
model.fit(train[features], train[target])
proba = model.predict_proba(train[features])
print(roc_auc_score(train[target], proba[:,1]))

0.931321179682


## 7. Кросс-валидация модели

In [35]:
cross_auc = cross_val_score(model, train[features], train[target], cv=kf, scoring='roc_auc')
print(np.mean(cross_auc))
print(cross_auc)

0.919870358855
[ 0.91422048  0.91389477  0.92544073  0.92665222  0.9191436 ]


## 8. Определение важности признаков

In [36]:
importances = model.feature_importances_
std = np.std([model.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
indices = indices.tolist()

# Print the feature ranking
print("Feature ranking:\n")

for f in indices:
    print("feature ", features[f], "(%f)" % (importances[f]))

Feature ranking:

feature  user.followers_count (0.420719)
feature  is_reply (0.263027)
feature  user.listed_count (0.155592)
feature  user.statuses_count (0.066235)
feature  user_reg_days (0.023423)
feature  count_reference (0.021346)
feature  nlp_prediction (0.011733)
feature  user.favourites_count (0.008051)
feature  count_url (0.006430)
feature  user.is_translation_enabled (0.004755)
feature  user.friends_count (0.004254)
feature  user_discr_len (0.004241)
feature  text_len (0.002740)
feature  clear_text_with_tags_len (0.002052)
feature  clear_text_len (0.001918)
feature  user.utc_offset (0.001801)
feature  count_hashteg (0.000716)
feature  is_en (0.000560)
feature  user.geo_enabled (0.000205)
feature  is_claim (0.000135)
feature  is_question (0.000066)


## 9. Построение и запись прогноза для тестового датасета

In [37]:
test['nlp_prediction'] = test['clear_text_with_tags'].apply(classify)

Строим новый прогноз и обновляем вероятности того, что ретвитов больше 20.

In [38]:
proba = model.predict_proba(test[features])
test['probability'] = proba[:,1]

prediction = pd.DataFrame(test['probability'])
prediction.head()

,probability
id,
629692042952765440,0.012906
629692042717855745,0.013750
629692039974813696,0.012951
629692038242566145,0.988021
629692036879413248,0.016566


Записываем полученный результат в файл **prediction.csv**.

In [39]:
prediction.to_csv('prediction.csv')